In [1]:
import csv
import os
import re
from collections import defaultdict
from datetime import datetime, timedelta

In [2]:
# CSVファイルが保存されているディレクトリのパスを設定してください
csv_directory = 'data'  # ここを実際のパスに変更

# チーム名と team_id のマッピングを手動で設定
team_name_to_id = {
    'アトムズ': 1,
    'サンケイアトムズ': 1,
    'サンケイスワローズ': 1,
    'ヤクルトアトムズ': 1,
    'ヤクルトスワローズ': 1,
    '国鉄スワローズ': 1,
    '東京ヤクルトスワローズ': 1,
    '読売ジャイアンツ': 2,
    '中日ドラゴンズ': 3,
    '名古屋ドラゴンズ': 3,
    '大阪タイガース': 4,
    '阪神タイガース': 4,
    '広島カープ': 5,
    '広島東洋カープ': 5,
    '大洋ホエールズ': 6,
    '大洋松竹ロビンス': 6,
    '横浜DeNAベイスターズ': 6,
    '横浜ベイスターズ': 6,
    '横浜大洋ホエールズ': 6,
    '松竹ロビンス': 7,
    '西日本パイレーツ': 8,
    # 必要に応じて他のチームを追加
}

In [15]:
# チーム履歴を保持する辞書
team_history = defaultdict(list)

# ディレクトリ内のCSVファイルを取得
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('central.csv')]

In [16]:
for filename in csv_files:
    csv_path = os.path.join(csv_directory, filename)
    # 年度をファイル名から抽出（例: '2020.csv' から '2020' を取得）
    match = re.search(r'standings_(\d{4})_central\.csv', filename)
    year = match.group(1)
    try:
        year_int = int(year)
    except ValueError:
        print(f"ファイル名から年度を取得できませんでした: {filename}")
        continue

    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            team_name = row['チーム'].strip()
            # チーム名がマッピングに存在するか確認
            if team_name not in team_name_to_id:
                print(f"未定義のチーム名が見つかりました: {team_name} (ファイル: {filename})")
                continue

            team_id = team_name_to_id[team_name]

            # 開始日と終了日を設定（年度の1月1日から12月31日まで）
            start_date = datetime(year_int, 1, 1).date()
            end_date = datetime(year_int, 12, 31).date()

            # チーム履歴に追加
            team_history[team_id].append({
                'team_name': team_name,
                'start_date': start_date,
                'end_date': end_date,
                'city': None,  # 必要に応じて設定
                'league': 'セントラル・リーグ',
                'division': None,
                'stadium_name': None  # 必要に応じて設定
            })

In [17]:
# チーム履歴の統合（連続する同じチーム名を一つの期間にまとめる）
final_team_history = []

for team_id, histories in team_history.items():
    # 開始日でソート
    histories.sort(key=lambda x: x['start_date'])
    merged_histories = []
    prev_entry = histories[0]

    for entry in histories[1:]:
        # チーム名が同じで期間が連続している場合は統合
        if (entry['team_name'] == prev_entry['team_name'] and
            entry['start_date'] == prev_entry['end_date'] + timedelta(days=1)):
            # 終了日を更新
            prev_entry['end_date'] = entry['end_date']
        else:
            merged_histories.append(prev_entry)
            prev_entry = entry
    merged_histories.append(prev_entry)

    # 統合結果を保存
    for history in merged_histories:
        final_team_history.append({
            'team_id': team_id,
            **history  # チーム名や期間など
        })

# TeamHistoryのデータをCSVで出力
output_csv = 'output/team_history_central_output.csv'

with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['team_id', 'start_date', 'end_date', 'team_name', 'city', 'league', 'division', 'stadium_name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for entry in final_team_history:
        writer.writerow({
            'team_id': entry['team_id'],
            'start_date': entry['start_date'].strftime('%Y-%m-%d'),
            'end_date': entry['end_date'].strftime('%Y-%m-%d') if entry['end_date'] else '',
            'team_name': entry['team_name'],
            'city': entry['city'] if entry['city'] else '',
            'league': entry['league'] if entry['league'] else '',
            'division': entry['division'] if entry['division'] else '',
            'stadium_name': entry['stadium_name'] if entry['stadium_name'] else ''
        })

print(f"TeamHistoryのデータをCSVファイルに出力しました: {output_csv}")

TeamHistoryのデータをCSVファイルに出力しました: output/team_history_output.csv


In [3]:
# パリーグ

# チーム名と team_id のマッピングを手動で設定
team_name_to_id = {
    'オリックス・バファローズ': 9,
    'オリックス・ブルーウェーブ': 9,
    'オリックス・ブレーブス': 9,
    '阪急ブレーブス': 9,
    '南海ホークス': 10,
    '福岡ソフトバンクホークス': 10,
    '福岡ダイエーホークス': 10,
    'クラウンライター・ライオンズ': 11,
    '埼玉西武ライオンズ': 11,
    '太平洋クラブ・ライオンズ': 11,
    '西武ライオンズ': 11,
    '西鉄クリッパース': 11,
    '西鉄ライオンズ': 11,
    'ロッテ・オリオンズ': 12,
    '千葉ロッテマリーンズ': 12,
    '東京オリオンズ': 12,
    '毎日オリオンズ': 12,
    '毎日大映オリオンズ': 12,
    '北海道日本ハムファイターズ': 13,
    '日拓ホーム・フライヤーズ': 13,
    '日本ハム・ファイターズ': 13,
    '東急フライヤーズ': 13,
    '東映フライヤーズ': 13,
    '東北楽天ゴールデンイーグルス': 14,
    '大阪近鉄バファローズ': 15,
    '近鉄バファロー': 15,
    '近鉄バファローズ': 15,
    '近鉄パールス': 15,
    'トンボユニオンズ': 16,
    '高橋ユニオンズ': 16,
    '大映スターズ': 17,
    '大映ユニオンズ': 17,
    # 必要に応じて他のチームを追加
}

# チーム履歴を保持する辞書
team_history = defaultdict(list)

# ディレクトリ内のCSVファイルを取得
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('pacific.csv')]

for filename in csv_files:
    csv_path = os.path.join(csv_directory, filename)
    # 年度をファイル名から抽出（例: '2020.csv' から '2020' を取得）
    match = re.search(r'standings_(\d{4})_pacific\.csv', filename)
    year = match.group(1)
    try:
        year_int = int(year)
    except ValueError:
        print(f"ファイル名から年度を取得できませんでした: {filename}")
        continue

    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            team_name = row['チーム'].strip()
            # チーム名がマッピングに存在するか確認
            if team_name not in team_name_to_id:
                print(f"未定義のチーム名が見つかりました: {team_name} (ファイル: {filename})")
                continue

            team_id = team_name_to_id[team_name]

            # 開始日と終了日を設定（年度の1月1日から12月31日まで）
            start_date = datetime(year_int, 1, 1).date()
            end_date = datetime(year_int, 12, 31).date()

            # チーム履歴に追加
            team_history[team_id].append({
                'team_name': team_name,
                'start_date': start_date,
                'end_date': end_date,
                'city': None,  # 必要に応じて設定
                'league': 'パシフィック・リーグ',
                'division': None,
                'stadium_name': None  # 必要に応じて設定
            })

# チーム履歴の統合（連続する同じチーム名を一つの期間にまとめる）
final_team_history = []

for team_id, histories in team_history.items():
    # 開始日でソート
    histories.sort(key=lambda x: x['start_date'])
    merged_histories = []
    prev_entry = histories[0]

    for entry in histories[1:]:
        # チーム名が同じで期間が連続している場合は統合
        if (entry['team_name'] == prev_entry['team_name'] and
            entry['start_date'] == prev_entry['end_date'] + timedelta(days=1)):
            # 終了日を更新
            prev_entry['end_date'] = entry['end_date']
        else:
            merged_histories.append(prev_entry)
            prev_entry = entry
    merged_histories.append(prev_entry)

    # 統合結果を保存
    for history in merged_histories:
        final_team_history.append({
            'team_id': team_id,
            **history  # チーム名や期間など
        })

# TeamHistoryのデータをCSVで出力
output_csv = 'output/team_history_pacific_output.csv'

with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['team_id', 'start_date', 'end_date', 'team_name', 'city', 'league', 'division', 'stadium_name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for entry in final_team_history:
        writer.writerow({
            'team_id': entry['team_id'],
            'start_date': entry['start_date'].strftime('%Y-%m-%d'),
            'end_date': entry['end_date'].strftime('%Y-%m-%d') if entry['end_date'] else '',
            'team_name': entry['team_name'],
            'city': entry['city'] if entry['city'] else '',
            'league': entry['league'] if entry['league'] else '',
            'division': entry['division'] if entry['division'] else '',
            'stadium_name': entry['stadium_name'] if entry['stadium_name'] else ''
        })

print(f"TeamHistoryのデータをCSVファイルに出力しました: {output_csv}")

TeamHistoryのデータをCSVファイルに出力しました: output/team_history_pacific_output.csv
